In [86]:
import numpy as np
import pandas as pd
%matplotlib inline
import glob
import keras
from functools import reduce
from sklearn import preprocessing
from hrvanalysis import remove_outliers, remove_ectopic_beats, interpolate_nan_values
from hrvanalysis import get_time_domain_features

In [87]:
path = r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Windowed Data' # use your path
all_files = glob.glob(path + "/*.csv")

In [88]:
li=[]
frame=[]

In [ ]:
##This part is for normalizing and standardizing windows. Normalizing does not make sence because it makes all
##minimumes to 0 and all maximumes to 1


for filename in all_files:
    df1 = pd.read_csv(filename, index_col=None, header=0)
    df1['hr']= df1['hr'].astype(float)
   
    
    df2=df1[['hr','windowno', 'ptsd_moment']]
    hrmean=df2.groupby('windowno', as_index=False)['hr'].mean()
    hrmean=hrmean.rename(columns={"hr": "hrmean"})
    #hrmin=df2.groupby('windowno', as_index=False)['hr'].min()
    #hrmin=hrmin.rename(columns={"hr": "hrmin"})
    #hrmax=df2.groupby('windowno', as_index=False)['hr'].max()
    #hrmax=hrmax.rename(columns={"hr": "hrmax"})
    hrsd=df2.groupby('windowno').agg(np.std,ddof=0)['hr']
    hrsd=pd.DataFrame(hrsd)
    hrsd=hrsd.rename(columns={"hr": "hrsd"})
    merged= pd.merge(hrmean, hrsd, on='windowno', how='inner')
    merged= pd.merge(df2,merged, on='windowno', how='inner')
    #merged= pd.merge(hrmin,merged, on='windowno', how='inner')
    #merged= pd.merge(hrmax,merged, on='windowno', how='inner')
    stress=df2.groupby('windowno', as_index=False)['ptsd_moment'].max()
    #normalize
    #merged['nhr']=(merged['hr']-merged['hrmin'])/(merged['hrmax']-merged['hrmin'])
    merged['zhr']=(merged['hr']-merged['hrmean'])/(merged['hrsd'])
    
    
    zhrmean=merged.groupby('windowno', as_index=False)['zhr'].mean()
    zhrmean=zhrmean.rename(columns={"zhr": "zhrmean"})
    zhrmin=merged.groupby('windowno', as_index=False)['zhr'].min()
    zhrmin=zhrmin.rename(columns={"zhr": "zhrmin"})
    zhrmax=merged.groupby('windowno', as_index=False)['zhr'].max()
    zhrmax=zhrmax.rename(columns={"zhr": "zhrmax"})
    zhrsd=merged.groupby('windowno').agg(np.std,ddof=0)['zhr']
    zhrsd=pd.DataFrame(zhrsd)
    zhrsd=zhrsd.rename(columns={"zhr": "zhrsd"})
    
    
    
    
    participant = df1['participant'].iloc[0]
    data=reduce(lambda x,y: pd.merge(x,y, on='windowno', how='outer'),
       [hrmean,hrmin,hrmax,hrsd,stress])
    data['participant']= participant
    li.append(data)
frame = pd.concat(li, axis=0, ignore_index=True)

In [89]:
li=[]
framehrv=pd.DataFrame()

In [90]:
######This is the important part where you extract HRV features from HR along with HR features


for filename in all_files:
    df1 = pd.read_csv(filename, index_col=None, header=0)
    df1['hr']= df1['hr'].astype(float)
    
    i=0
    
    li=[]
    data1=df1[['hr','windowno', 'ptsd_moment']]
    #stress=data1.groupby('windowno', as_index=False)['ptsd_moment'].max()
    data1['hrv'] = 60000/data1['hr']
    
    windowno=data1['windowno']
    
    for i in windowno:
        d2=data1.loc[data1['windowno'] == i] 
        time_domain_features = get_time_domain_features(d2['hrv'])
        stress=d2['ptsd_moment'].max()
        time_domain_features.update( {'stress' : stress} )
        time_domain_features.update( {'windowno' : i} )
        li.append(time_domain_features)
    Hrfeatures=pd.DataFrame.from_dict(li)
    Hrfeatures=Hrfeatures.drop_duplicates()  
    Hrfeatures['participant']=df1['participant'].iloc[0]
    f=[framehrv,Hrfeatures]
    framehrv=pd.concat(f)


//miniconda3/envs/PTSD/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [91]:
frame1=framehrv

In [92]:
frame1['hrrange']= frame1['max_hr']-frame1['min_hr']

In [94]:
fr=frame1.drop_duplicates()  

In [96]:
fr.to_csv (r'/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/HR Features/HR and HRV Features Correct_dataframe.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

In [93]:
frame1

,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,stress,windowno,participant,hrrange
0,901.300934,41.260657,15.314909,1,1.666667,6,10.000000,15.322961,895.522388,145.824127,0.017001,0.045779,66.708347,73.0,62.0,3.040741,1.0,2914,13,11.0
60,838.351222,54.519109,6.919900,0,0.000000,1,1.666667,7.614755,851.106640,187.500000,0.009083,0.065031,71.868832,80.0,64.0,4.653403,1.0,2915,13,16.0
120,700.978066,23.307441,5.769311,0,0.000000,1,2.000000,5.822111,697.674419,72.366658,0.008306,0.033250,85.688151,91.0,82.0,2.840909,1.0,2917,13,9.0
170,676.615466,17.752883,5.051157,0,0.000000,0,0.000000,5.051157,674.157303,53.708440,0.007465,0.026238,88.736465,92.0,85.0,2.299090,1.0,2918,13,7.0
230,713.850011,40.319575,3.397830,0,0.000000,0,0.000000,4.581899,701.778386,119.880120,0.006419,0.056482,84.311374,91.0,77.0,4.667418,0.0,2922,13,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3482,966.289428,40.660031,12.717445,0,0.000000,6,11.764706,12.740265,983.606557,107.858243,0.013185,0.042079,62.202333,66.0,59.0,2.620253,0.0,10624,3,7.0
3533,953.496975,39.365731,13.197977,1,1.666667,4,6.666667,13.219671,952.380952,138.960371,0.013864,0.041286,63.031590,67.0,58.0,2.574690,0.0,10625,3,9.0
3593,544.572675,11.348862,2.583908,0,0.000000,0,0.000000,2.656126,540.540541,40.455120,0.004877,0.020840,110.224178,113.0,105.0,2.229094,0.0,11159,3,8.0
3651,631.098070,17.112727,3.716191,0,0.000000,0,0.000000,3.762925,625.000000,47.095761,0.005963,0.027116,95.140097,98.0,91.0,2.525024,0.0,11413,3,7.0


In [95]:
fr

,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,stress,windowno,participant,hrrange
0,901.300934,41.260657,15.314909,1,1.666667,6,10.000000,15.322961,895.522388,145.824127,0.017001,0.045779,66.708347,73.0,62.0,3.040741,1.0,2914,13,11.0
60,838.351222,54.519109,6.919900,0,0.000000,1,1.666667,7.614755,851.106640,187.500000,0.009083,0.065031,71.868832,80.0,64.0,4.653403,1.0,2915,13,16.0
120,700.978066,23.307441,5.769311,0,0.000000,1,2.000000,5.822111,697.674419,72.366658,0.008306,0.033250,85.688151,91.0,82.0,2.840909,1.0,2917,13,9.0
170,676.615466,17.752883,5.051157,0,0.000000,0,0.000000,5.051157,674.157303,53.708440,0.007465,0.026238,88.736465,92.0,85.0,2.299090,1.0,2918,13,7.0
230,713.850011,40.319575,3.397830,0,0.000000,0,0.000000,4.581899,701.778386,119.880120,0.006419,0.056482,84.311374,91.0,77.0,4.667418,0.0,2922,13,14.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3482,966.289428,40.660031,12.717445,0,0.000000,6,11.764706,12.740265,983.606557,107.858243,0.013185,0.042079,62.202333,66.0,59.0,2.620253,0.0,10624,3,7.0
3533,953.496975,39.365731,13.197977,1,1.666667,4,6.666667,13.219671,952.380952,138.960371,0.013864,0.041286,63.031590,67.0,58.0,2.574690,0.0,10625,3,9.0
3593,544.572675,11.348862,2.583908,0,0.000000,0,0.000000,2.656126,540.540541,40.455120,0.004877,0.020840,110.224178,113.0,105.0,2.229094,0.0,11159,3,8.0
3651,631.098070,17.112727,3.716191,0,0.000000,0,0.000000,3.762925,625.000000,47.095761,0.005963,0.027116,95.140097,98.0,91.0,2.525024,0.0,11413,3,7.0


In [53]:
df1=pd.read_csv('/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/Windowed Data/Windowed Participant6.csv')

In [54]:
df1['hr']= df1['hr'].astype(float)

In [55]:
data1=df1[['hr','windowno', 'ptsd_moment']]
    

In [56]:
data1['hrv'] = 60000/data1['hr']

//miniconda3/envs/PTSD/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [82]:
i=0
li=[]
windowno=data1['windowno']

In [83]:
#for i in data1['windowno']:
for i in windowno:
    d2=data1.loc[data1['windowno'] == i] 
    time_domain_features = get_time_domain_features(d2['hrv'])
    stress=d2['ptsd_moment'].max()
    time_domain_features.update( {'stress' : stress} )
    time_domain_features.update( {'windowno' : i} )
    li.append(time_domain_features)

Hrfeatures=pd.DataFrame.from_dict(li)
Hrfeatures=Hrfeatures.drop_duplicates()  
Hrfeatures['participant']=df1['participant'].iloc[0]

#f=[framehrv,Hrfeatures]
#framehrv=pd.concat(f)

In [85]:
Hrfeatures

,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,stress,windowno,participant
0,773.988767,24.339283,11.448568,1,1.666667,4,6.666667,11.484572,779.220779,127.450980,0.014838,0.031447,77.595833,85.0,72.0,2.418717,0,993,6
60,754.729419,35.916218,5.182992,0,0.000000,0,0.000000,5.329964,750.000000,121.155638,0.007062,0.047588,79.675000,87.0,74.0,3.740627,0,994,6
120,830.470417,40.838445,12.950681,0,0.000000,5,10.638298,13.053421,821.917808,181.236674,0.015718,0.049175,72.425532,84.0,67.0,3.653880,0,995,6
167,786.464285,40.757755,6.072236,0,0.000000,0,0.000000,6.784095,800.000000,159.468439,0.008626,0.051824,76.500000,86.0,70.0,4.082963,0,997,6
218,766.533480,42.304177,5.464074,0,0.000000,0,0.000000,6.480478,750.000000,142.857143,0.008454,0.055189,78.500000,84.0,70.0,4.140393,0,1000,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6360,836.567363,46.551545,10.444725,0,0.000000,4,6.666667,10.456460,833.333333,159.090909,0.012499,0.055646,71.941667,80.0,66.0,3.991754,0,3434,6
6420,826.690903,48.646210,10.654575,0,0.000000,4,6.666667,10.666080,833.333333,173.076923,0.012902,0.058844,72.825000,80.0,65.0,4.230667,0,3435,6
6480,815.998485,74.816821,9.720913,0,0.000000,4,6.666667,10.101551,789.473684,214.608434,0.012379,0.091687,74.133333,83.0,64.0,6.650982,0,3486,6
6540,751.482084,66.492374,9.665270,0,0.000000,3,5.000000,9.834944,774.225774,227.853987,0.013087,0.088482,80.479167,93.5,69.0,7.278721,0,6242,6


In [80]:
windowno

<bound method IndexOpsMixin.tolist of 0        993
1        993
2        993
3        993
4        993
        ... 
6635    6352
6636    6352
6637    6352
6638    6352
6639    6352
Name: windowno, Length: 6640, dtype: int64>

In [64]:
Hrfeatures.drop_duplicates() 

,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,stress,windowno,participant
0,901.300934,41.260657,15.314909,1,1.666667,6,10.000000,15.322961,895.522388,145.824127,0.017001,0.045779,66.708347,73.0,62.0,3.040741,1.0,2914,6
60,838.351222,54.519109,6.919900,0,0.000000,1,1.666667,7.614755,851.106640,187.500000,0.009083,0.065031,71.868832,80.0,64.0,4.653403,1.0,2915,6
120,700.978066,23.307441,5.769311,0,0.000000,1,2.000000,5.822111,697.674419,72.366658,0.008306,0.033250,85.688151,91.0,82.0,2.840909,1.0,2917,6
170,676.615466,17.752883,5.051157,0,0.000000,0,0.000000,5.051157,674.157303,53.708440,0.007465,0.026238,88.736465,92.0,85.0,2.299090,1.0,2918,6
230,713.850011,40.319575,3.397830,0,0.000000,0,0.000000,4.581899,701.778386,119.880120,0.006419,0.056482,84.311374,91.0,77.0,4.667418,0.0,2922,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
695432,966.289428,40.660031,12.717445,0,0.000000,6,11.764706,12.740265,983.606557,107.858243,0.013185,0.042079,62.202333,66.0,59.0,2.620253,0.0,10624,6
695483,953.496975,39.365731,13.197977,1,1.666667,4,6.666667,13.219671,952.380952,138.960371,0.013864,0.041286,63.031590,67.0,58.0,2.574690,0.0,10625,6
695543,544.572675,11.348862,2.583908,0,0.000000,0,0.000000,2.656126,540.540541,40.455120,0.004877,0.020840,110.224178,113.0,105.0,2.229094,0.0,11159,6
695601,631.098070,17.112727,3.716191,0,0.000000,0,0.000000,3.762925,625.000000,47.095761,0.005963,0.027116,95.140097,98.0,91.0,2.525024,0.0,11413,6


In [23]:
Hrfeatures.loc[Hrfeatures['windowno']==2914]

,mean_nni,sdnn,sdsd,nni_50,pnni_50,nni_20,pnni_20,rmssd,median_nni,range_nni,cvsd,cvnni,mean_hr,max_hr,min_hr,std_hr,stress,windowno,participant
0,901.300934,41.260657,15.314909,1,1.666667,6,10.000000,15.322961,895.522388,145.824127,0.017001,0.045779,66.708347,73.0,62.0,3.040741,1.0,2914,6
236063,544.515940,5.823281,1.635251,0,0.000000,0,0.000000,1.635766,545.454402,22.021940,0.003004,0.010694,110.202019,113.0,108.5,1.173046,0.0,2914,6
237701,904.632447,17.281674,20.567674,1,2.325581,5,11.627907,20.568843,909.090909,99.811467,0.022737,0.019104,66.349496,71.0,63.5,1.283496,0.0,2914,6
458041,900.566912,9.247179,4.901312,0,0.000000,0,0.000000,4.904451,900.814519,47.879617,0.005446,0.010268,66.631554,68.0,64.5,0.674293,0.0,2914,6


In [27]:
data1.loc[data1['windowno']==2914]

,hr,windowno,ptsd_moment,hrv


In [42]:
framehr=pd.read_csv('/Users/mahnooshsadeghi/Desktop/PTSD/PTSD Data/HR Features/HR Features_dataframe.csv')

In [52]:
framehr.loc[(framehr['participant']==6) & (framehr['windowno']==2914)]

,windowno,hrmean,hrmin,hrmax,hrsd,ptsd_moment,participant,hrrange


In [39]:
data1.iloc[4700:4760]

,hr,windowno,ptsd_moment,hrv
4700,68.000000,2048,0,882.352941
4701,66.000000,2048,0,909.090909
4702,67.000000,2048,0,895.522388
4703,66.000000,2048,0,909.090909
4704,65.000000,2048,0,923.076923
4705,64.000000,2048,0,937.500000
4706,63.833333,2048,0,939.947781
4707,63.666667,2048,0,942.408377
4708,63.500000,2048,0,944.881890
4709,63.333333,2048,0,947.368421


In [ ]:
df1['nhr']=(df1['hr']-df1['hr'].mean())/df1['hr'].std()

In [ ]:
df2=df1[['hr','windowno', 'ptsd_moment']]

In [ ]:
hrmean=df2.groupby('windowno', as_index=False)['hr'].mean()
hrmean=hrmean.rename(columns={"hr": "hrmean"})
hrmin=df2.groupby('windowno', as_index=False)['hr'].min()
hrmin=hrmin.rename(columns={"hr": "hrmin"})
hrmax=df2.groupby('windowno', as_index=False)['hr'].max()
hrmax=hrmax.rename(columns={"hr": "hrmax"})
hrsd=df1.groupby('windowno').agg(np.std,ddof=0)['hr']
hrsd=pd.DataFrame(hrsd)
hrsd=hrsd.rename(columns={"hr": "hrsd"})

In [ ]:
merged= pd.merge(hrmax, hrmin, on='windowno', how='inner')
merged= pd.merge(hrsd,merged, on='windowno', how='inner')
merged= pd.merge(hrmean,merged, on='windowno', how='inner')
merged= pd.merge(df2,merged, on='windowno', how='inner')

In [ ]:
merged['nhr']=(merged['hr']-merged['hrmin'])/(merged['hrmax']-merged['hrmin'])

In [ ]:
merged['zhr']=(merged['hr']-merged['hrmean'])/(merged['hrsd'])

In [ ]:
merged.iloc[:60]

In [ ]:
df2.isnull().sum()

In [ ]:
zhrmean=merged.groupby('windowno', as_index=False)['zhr'].mean()
zhrmean
